# DVH Analysis

This notebook demonstrates how to compute and plot a DVH (Dose Volume Histogram).

## Import Modules

In [ ]:
try:
    import platipy
except:
    !pip install platipy
    import platipy

import matplotlib
import matplotlib.pyplot as plt
import SimpleITK as sitk

%matplotlib inline

from platipy.imaging.tests.data import get_hn_nifti
from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com
from platipy.imaging.dose.dvh import calculate_dvh_for_labels, calculate_d_x, calculate_v_x
from platipy.imaging.visualisation.dose import visualise_dose

## Download Test Data

This will download some data from the TCIA TCGA-HNSC dataset. The data is for one patient and
contains a CT, dose and some structures.

In [ ]:
data_path = get_hn_nifti()

## Load data

Let's read in the data that we've downloaded

In [ ]:
test_pat_path = data_path.joinpath("TCGA_CV_5977")

ct_image = sitk.ReadImage(str(test_pat_path.joinpath("IMAGES/TCGA_CV_5977_1_CT_ONC_NECK_NECK_4.nii.gz")))

dose = sitk.ReadImage(str(test_pat_path.joinpath("DOSES/TCGA_CV_5977_1_PLAN.nii.gz")))

dose = sitk.Resample(dose, ct_image)

structure_names =["BRAINSTEM", "MANDIBLE", "CTV_60_GY", "PTV60", "CORD", "L_PAROTID", "R_PAROTID"]

structures = {
    s: sitk.ReadImage(str(test_pat_path.joinpath("STRUCTURES", f"TCGA_CV_5977_1_RTSTRUCT_{s}.nii.gz"))) for s in structure_names
}

## Visualise data

and now let's visualise the data we've got

In [ ]:
vis = ImageVisualiser(ct_image, cut=get_com(structures["PTV60"]))

vis.add_scalar_overlay(dose, discrete_levels=20, colormap=matplotlib.colormaps.get_cmap("inferno"), name="Dose (Gy)")
vis.add_contour(structures)

fig = vis.show()

## Compute DVH

here we compute the DVH using the dose and structures loaded. We get the DVH back in a pandas
DataFrame object.

In [ ]:
dvh = calculate_dvh_for_labels(dose, structures)

In [ ]:
dvh

## Plot DVH

using the pandas DataFrame, we plot the DVH here. The DVH first needs to be reshaped to prepare it for plotting.

In [ ]:
# Reshape the DVH
plt_dvh = dvh
plt_dvh = plt_dvh.set_index("label")
plt_dvh = plt_dvh.iloc[:,3:].transpose()

# Plot the DVH
fig, ax = plt.subplots()
plt_dvh.plot(ax=ax, kind="line", colormap=matplotlib.colormaps.get_cmap("rainbow"), legend=False)

# Add labels and show plot
plt.legend(loc='best')
plt.xlabel("Dose (Gy)")
plt.ylabel("Frequency")
plt.title("Dose Volume Histogram (DVH)")
plt.show()

## DVH Metrics

Finally, we extract commonly used metrics from the DVH. In the following cells we extract the D0 and D95 as well as the V5 and V20.

In [ ]:
df_metrics_d = calculate_d_x(dvh, [0, 95])
df_metrics_d

In [ ]:
df_metrics_v = calculate_v_x(dvh, [5, 20])
df_metrics_v

## Dose and DVH visualisation

The `visualise_dose` function can produce a visualisation including the DVH and dose metrics.

In [ ]:
fig, df_metrics = visualise_dose(
    ct_image,
    dose,
    structures,
    dvh=dvh,
    d_points=[0, 95],
    v_points=[5],
    d_cc_points=[10],
    structure_for_limits=dose>5,
    expansion_for_limits=40,
    contour_cmap=matplotlib.colormaps.get_cmap("rainbow"),
    dose_cmap=matplotlib.colormaps.get_cmap("inferno"),
    title="TCGA_CV_5977 Dose Metrics")